# Видобуток та збагачення структурованих даних

### ТЕОРЕТИЧНА ЧАСТИНА ТА ПРИКЛАДИ

Для роботи з структурованими табличними даними, які розміщені в реляційній БД [SQLite](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiUyMOv1orvAhWDK3cKHac9DyMQFjAAegQIBxAD&url=https%3A%2F%2Fru.wikipedia.org%2Fwiki%2FSQLite&usg=AOvVaw1s-RMK2VC2tXyDZxZXDYDo) в Python передбачена стандартна бібліотека [sqlite3](https://docs.python.org/3/library/sqlite3.html)

In [1]:
# підключення бібліотеки
import sqlite3

#### ПРИКЛАД ВИЛУЧЕННЯ ДАНИХ З БД
Створити БД SQLite з ім'ям `FIT-4m`, та імпорувати в неї таблицю, що містить журнал групи `jornal`

In [3]:
# створюємо з'єднання з БД
conn = sqlite3.connect("FIT-4m.db")

[імпорт csv файла в sqlite через CLI](https://www.sqlitetutorial.net/sqlite-import-csv/) 

In [4]:
# створюємо об'єкт `cursor`, що відповідає за реалізацію операцій з таблицями БД
cur = conn.cursor()

In [5]:
print(type(cur), end='\n\n')
print(dir(cur))

<class 'sqlite3.Cursor'>

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'arraysize', 'close', 'connection', 'description', 'execute', 'executemany', 'executescript', 'fetchall', 'fetchmany', 'fetchone', 'lastrowid', 'row_factory', 'rowcount', 'setinputsizes', 'setoutputsize']


##### виведемо ВСІ записи таблиці `jornal`

In [7]:
# створимо інстанс для операції `SELECT`
students_cur = cur.execute("SELECT * FROM jornal")

# будуємо список записів за допомогою метода `fetchall()`
students_list = students_cur.fetchmany(3)

_ = [print(x) for x in students_list]

('Антонов', 'Іван', 'i.antonov_fit_4m_22_m_d@knute.edu.ua', '')
('Антонюк', 'Данило', 'd.antonyuk_fit_4m_22_m_d@knute.edu.ua', '')
('Багінський', 'Вадим', 'v.bahinskyy_fit_4m_22_m_d@knute.edu.ua', '')


#### ПРИКЛАД ЗБАГАЧЕННЯ ДАНИХ З ЗОВНІШНЬОГО ДЖЕРЕЛА

Додати до списка студентів їх оцінки за вступне тестування, які розташовані в csv файлі `test_result.csv`

імпортуємо `test_result.csv` в робочу БД як таблицю `test`

[імпорт csv файла в sqlite через CLI](https://www.sqlitetutorial.net/sqlite-import-csv/) 

##### Cтворимо нову таблицю, яка буде містити вміст таблиці `jornal` та поле оцінок з таблиці `test`



In [8]:
# створимо вираз на побудову пустої таблиці `jornal_test` з відповідними полями
sql_stmt = "CREATE TABLE IF NOT EXISTS jornal_test \
              (Surname TEXT,         \
               Name TEXT,            \
               Email TEXT,           \
               GitHub TEXT,          \
               Test_result INT);"

# виконаемо запит на створення таблиці
cur.execute(sql_stmt)

In [9]:
# перевіримо стан виконання операції - виведемо список таблиць
tables_list = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables_list.fetchall()

[('jornal',), ('test_result',), ('jornal_test',)]

In [10]:
# створимо запит на об'єднання таблиць 'jornal' та 'test' по прізвищу студента
sql_stmt = "SELECT jornal.*, test_result.test FROM  \
            jornal LEFT JOIN test_result ON jornal.Surname = test_result.Surname "

In [11]:
# для перевірки виконаємо і виведемо результати запиту
cur.execute(sql_stmt).fetchmany(7)

[('Антонов', 'Іван', 'i.antonov_fit_4m_22_m_d@knute.edu.ua', '', None),
 ('Антонюк', 'Данило', 'd.antonyuk_fit_4m_22_m_d@knute.edu.ua', '', None),
 ('Багінський', 'Вадим', 'v.bahinskyy_fit_4m_22_m_d@knute.edu.ua', '', 63),
 ('Бай',
  'Андрій',
  'a.bay_fit_4m_22_m_d@knute.edu.ua',
  'https://github.com/Ullrvetr',
  51),
 ('Безсмертний',
  'Володимир',
  'v.bezsmertnyy_fit_4m_22_m_d@knute.edu.ua',
  '',
  None),
 ('Будяну',
  'Максим',
  'm.budyanu_fit_4m_22_m_d@knute.edu.ua',
  'https://github.com/Budeanu',
  None),
 ('Власенко', 'Олександр', 'o.vlasenko_fit_4m_22_m_d@knute.edu.ua', '', 52)]

In [12]:
# заповнимо пусту таблицю результатми  запиту на злиття
_ = cur.execute("INSERT INTO jornal_test " + sql_stmt)

In [ ]:
# збережемо вміст нової таблиці в список
jornal_test_list = cur.execute("SELECT * FROM jornal_test").fetchall()

In [19]:
_ = [print(x) for x in jornal_test_list]

('Антонов', 'Іван', 'i.antonov_fit_4m_22_m_d@knute.edu.ua', '', None)
('Антонюк', 'Данило', 'd.antonyuk_fit_4m_22_m_d@knute.edu.ua', '', None)
('Багінський', 'Вадим', 'v.bahinskyy_fit_4m_22_m_d@knute.edu.ua', '', 63)
('Бай', 'Андрій', 'a.bay_fit_4m_22_m_d@knute.edu.ua', 'https://github.com/Ullrvetr', 51)
('Безсмертний', 'Володимир', 'v.bezsmertnyy_fit_4m_22_m_d@knute.edu.ua', '', None)
('Будяну', 'Максим', 'm.budyanu_fit_4m_22_m_d@knute.edu.ua', 'https://github.com/Budeanu', None)
('Власенко', 'Олександр', 'o.vlasenko_fit_4m_22_m_d@knute.edu.ua', '', 52)
('Гелла', 'Всеволод', 'v.hella_fit_4m_22_m_d@knute.edu.ua', '', 82)
('Гонгало', 'Вадим', 'v.honhalo_fit_4m_22_m_d@knute.edu.ua', 'https://github.com/Vaditos/DM--honhalo-', 44)
('Дринь', 'Ярослав', 'y.dryn_fit_4m_22_m_d@knute.edu.ua', '', None)
('Дубовий', 'Іван', 'i.dubovyy_fit_4m_22_m_d@knute.edu.ua', '', None)
('Копотун', 'Світлана', 's.kopotun_fit_4m_22_m_d@knute.edu.ua', 'https://github.com/Kopotuns', 44)
('Коротких', 'Віталій', 'v

In [16]:
# завершимо всі транзакції
conn.commit()

# закріємо з'єднання
conn.close()

#### ПРИКЛАД  ДЕСКРИПТИВНОГО АНАЛІЗУ ДАНИХ

Провести [попередній аналіз](https://ru.wikipedia.org/wiki/Описательная_статистика) отриманих даних з ціллю виявленя відхилень, помилок та інших непридатних даних.

Сведемо результати аналізу в таблицю, яка має наступний вигляд:

№| Показчик | Значення
:--:|:-------|-------:
1| кількість спостережень | xx 
2| кількість пустих значень | xx
4| середній бал|  xx.x
5| максимальний бал | xx
6| мінімальний бал  | xx
7| стандартне відхилення | xx.x
8| розмах вариації  | xx


In [17]:
# підключимо бібліотеку 'numpy' і дамо їй аліас 'np'
import numpy as np

ознайомитись з призначенням, можливостями та основним функціями [бібліотеки numpy](https://numpy.org) 

In [20]:
# перетворимо список студетів в numpay матрицю
jornal_test_arr = np.array(jornal_test_list)

In [21]:
print(type (jornal_test_arr), jornal_test_arr.shape)

<class 'numpy.ndarray'> (58, 5)


In [22]:
np.delete?

In [23]:
# створимо масив с результатами тестів 
test_result = np.delete(jornal_test_arr, [1,2,3], axis=1)

In [24]:
# Значення 'None' - не є числом, то заміним його на 0
test_result[test_result == None] = 0

In [25]:
# залишити тільки статистично важливі показчики
test_result = np.delete(test_result, 0, axis=1)

In [26]:
# будуємо словник з вихідними розрахунками
result_dict = {
    "кількість спостережень"   : len(test_result),
    "кількість пустих значень" : len(test_result) - np.count_nonzero(test_result),
    "середній бал"             : round(np.mean(test_result),1),
    "максимальний бал"         : np.max(test_result),
    "мінімальний бал"          : np.min(test_result[np.nonzero(test_result)]),
    "стандартне відхилення"    : round(np.std(test_result),1),
    "розмах вариації"          : np.max(test_result) - np.min(test_result[np.nonzero(test_result)]) 
    }

In [27]:
result_dict

{'кількість спостережень': 58,
 'кількість пустих значень': 22,
 'середній бал': 36.7,
 'максимальний бал': 82,
 'мінімальний бал': 35,
 'стандартне відхилення': 30.5,
 'розмах вариації': 47}

In [28]:
# вивести шапку
print(
"""
=======================================
№  : ПОКАЗЧИК               : ЗНАЧЕННЯ     
=======================================
"""      
      )

# вивести результати аналізу датасета
i = 1
for key, value in result_dict.items():
    print (f'{i:<3} {key:<25}  {value}')
    i += 1


№  : ПОКАЗЧИК               : ЗНАЧЕННЯ     

1   кількість спостережень     58
2   кількість пустих значень   22
3   середній бал               36.7
4   максимальний бал           82
5   мінімальний бал            35
6   стандартне відхилення      30.5
7   розмах вариації            47


### ІНДИВІДУАЛЬНЕ ЗАВДАННЯ

Користуючись результатами, що отримані в [Лабораторній роботі № 3](https://shkliarskiy.moodlecloud.com/mod/page/view.php?id=1193) виконати процедури видобутку, збагачення та попереднього аналізу даних.

__Постановка__: В 3-й лабораторній роботі отримано показчик, що характеризує окрему властивість квартири (ціна, метраж та ін.). 

Необхідно:
1. Відобразити цей показчик на адресу квартири, яка знаходиться в файлі `street_name.csv`, тобто побудувати новий файл `street_flat_propery.csv` якій містить 2 колонки: _<назва вулиці> <ваш показчик>_
2. На основі цьго файлу зробити відповідну таблицю в _SQLite_ та збагатити дані в цій таблиці назвою району де розташована квартира (файл `kiev_districts.csv`) 
3. Провести попередній аналіз отриманих даних - вивести первинні статистики показчика по районах (_середне_ , _мінімальне_ , _максимальне_ і т.д. - див. приклад вище)

In [ ]:
# Злиття файлу з показчиком та файлу з назвами вулиць в новий файл

YOUR_FILE = підставте ваш файл з Лаб №3...
with open(YOUR_FILE) as f1, \
     open('street_name.csv') as f2, \
     open('street_flat_propery.csv', 'w') as f3:
    
    for x, y in zip(f1, f2):
        line = f'{x[:-1].strip()},{y[:-1].strip()}\n'
        print(line)
        f3.write(line)

#### імпортувати в БД отриманий файл 'street_flat_propery.csv' через CLI
- __sqlite3 districts.db__ - запустити SQLite та відкрити БД
- __.mode csv__ - перевести БД для роботи з csv-файлами
- __.import street_flat_propery.csv street_flat_propery__ - створює таблицю _street_flat_propery_ та завантажує в неї вміст файлу _street_flat_propery.csv_
- __SELECT * FROM street_flat_propery LIMIT 10;__ - вивести 10 записів для перевірки
- __.quit__ - завершити роботу з CLI SQLite



In [ ]:
# Підключитись до БД 'districts.db' в якій є прив'язка вулиць до районів 
conn = напишить код ...

# створити відповідний курсор
cur  = напишить код ...

In [ ]:
# для перевірки виведіть 10 записів з таблиці 'street_district'
напишить код ...

In [ ]:
# побудувати запит на створення таблиці 'result' з 3-ма полями:
#     district - текстове
#     street - текстове
#     property - числове
stmt = напишить код ...

In [ ]:
# виконати запит
cur.execute(stmt)

In [ ]:
# перевірити схему таблиці
cur.execute ("SELECT name FROM PRAGMA_TABLE_INFO('result');").fetchall()

In [ ]:
# створити запит на злиття таблиці 'street_flat_propery' та 'street_district' 
# по полю 'street'
stmt_join = напишить код ...

In [ ]:
# перевірити запит
cur.execute(stmt_join).fetchall()

In [ ]:
# створити запит для наповнення таблиці `result` результатом запиту на об'єднання
stmt_insert = напишить код ..

In [ ]:
# виконати запит на заповнення
cur.execute(stmt_insert)

In [ ]:
# сберегти вміст створеної таблиці у вигляді списка
result_list = напишить код ...

#### для аналізу скористатися бібліотекою NumPy

In [ ]:
import numpy as np

In [ ]:
# створити np-масив з списку `result_list`
result_arr = напишить код ...

In [ ]:
# побудувати словник для результатів аналізу
result_dict = {
    "кількість спостережень"   : напишить код ... ,
    "середнє значення"         : напишить код ... ,
    "максимальне значення"     : напишить код ... ,
    "мінімальне значення"      : напишить код ... ,
    "стандартне відхилення"    : напишить код ... ,
    "розмах вариації"          : напишить код ...
    }

In [ ]:
# вивести шапку
print(
"""
================================================
№  :   РАЙОН        : ПОКАЗЧИК        : ЗНАЧЕННЯ     
================================================
"""      
      )

# вивести результати аналізу датасета
напишить код ...